In [116]:
import urllib.request
import bs4
import pandas as pd
import numpy as np
import pickle
import re
from tqdm.notebook import tqdm

def get_df(url):
    ad_url = 'https://www.letsrun.com/shoes/'
    titles = []
    posts = []
    original_post = []
    date = []
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers = {'User-Agent':user_agent,} 
    request = urllib.request.Request(url,None,headers)
    response = urllib.request.urlopen(request)
    webpage = response.read()
    soup = bs4.BeautifulSoup(webpage, 'html.parser')
    body = soup.find_all("ul", class_="forum-index-thread-list")
    entries = body[0].find_all('li', class_='thread')
    for i in entries:
        container = i.a
        if container is not None:
            post_url = container['href']
            if post_url != ad_url:
                title = container.renderContents().decode('utf-8').strip()
                post = i.findAll('span', class_='post-count')[0].text
                titles.append(title)
                posts.append(post)
                #request = urllib.request.Request(post_url,None,headers)
                #response = urllib.request.urlopen(request)
                #webpage = response.read()
                #post_soup = bs4.BeautifulSoup(webpage, 'html.parser')
                #print(post_soup.find_all("span", id="intelliTXT"))
                #original_post.append(post_body)
                   
    df = pd.DataFrame(list(zip(titles, posts)), columns =['title', 'posts', ])
    return df


def scrape_pages(num_pages):
    url_base = 'https://www.letsrun.com/forum?page='
    dfs = []
    for i in tqdm(range(num_pages)):
        idx = i + 1
        url = url_base + str(idx)
        dfs.append(get_df(url))    
    df_main = pd.concat(dfs).reset_index()
    return df_main

In [120]:
df = scrape_pages(50)
df

,index,title,posts
0,0,Chasing Master world records after 20 years .,294 posts
1,1,Are any Boulder runners NOT in a YouTube-d tra...,73 posts
2,2,My church was attacked,28 posts
3,3,"Shannon Rowbury 8:40 3k, Kerr 2:17 1k, Amos 1:...",51 posts
4,4,MICHAEL NORMAN 9.86!!!! (1.6),54 posts
...,...,...,...
3745,70,Lloyd Crowe,2 posts
3746,71,Sub 16 Help?,3 posts
3747,72,Microwaving shorts,12 posts
3748,73,LTHR: how can a 30 minute all-out test measure...,14 posts
